In [29]:
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException, TimeoutException
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.remote.webelement import WebElement

from tqdm import tqdm

import time, re, os
import pandas as pd

In [30]:
company_script = """var overviewItems = Array.from(document.querySelectorAll('.JobDetails_overviewItem__35s2T'));
var overviewData = {
    'Size': -1,
    'Founded': -1,
    'Type': -1,
    'Industry': -1,
    'Sector': -1,
    'Revenue': -1
};
overviewItems.forEach(function(item) {
    var label = item.querySelector('.JobDetails_overviewItemLabel__5vi0o');
    if (label) {
        var labelText = label.textContent.trim();
        if (overviewData.hasOwnProperty(labelText)) {
            var value = item.querySelector('.JobDetails_overviewItemValue__5TqNi');
            if (value) {
                value = value.textContent.trim()
                if (value != '--') {
                    overviewData[labelText] = value
                }
            }
        }
    }
});
return overviewData;
"""
easy_apply_script = """var applyContainer = document.querySelector('div.JobDetails_applyButtonContainer__Fzd66');
return applyContainer ? applyContainer.textContent.includes('Easy Apply') : false;
"""

In [31]:
us_states = [
    "Alabama", "Alaska", "Arizona", "Arkansas", "California", "Colorado", 
    "Connecticut", "Delaware", "Florida", "Georgia", "Hawaii", "Idaho", 
    "Illinois", "Indiana", "Iowa", "Kansas", "Kentucky", "Louisiana", 
    "Maine", "Maryland", "Massachusetts", "Michigan", "Minnesota", 
    "Mississippi", "Missouri", "Montana", "Nebraska", "Nevada", 
    "New Hampshire", "New Jersey", "New Mexico", "New York", 
    "North Carolina", "North Dakota", "Ohio", "Oklahoma", "Oregon", 
    "Pennsylvania", "Rhode Island", "South Carolina", "South Dakota", 
    "Tennessee", "Texas", "Utah", "Vermont", "Virginia", "Washington, DC", "Washington State",
    "West Virginia", "Wisconsin", "Wyoming"
]


In [32]:
def get_jobs(keyword, location=None, num_jobs=None, verbose=False):
    
    '''Gathers jobs as a dataframe, scraped from Glassdoor'''
    
    #Initializing the webdriver
    options = Options()
    # options = webdriver.ChromeOptions()
    # options = webdriver.EdgeOptions()
    options.page_load_strategy = 'normal'
    # options.add_experimental_option("detach", True)
    
    #Uncomment the line below if you'd like to scrape without a new Chrome window every time.
    # options.add_argument('headless')
    
    #Change the path to where chromedriver is in your home folder.
    # driver = webdriver.Chrome(executable_path="chromedriver", options=options)
    driver = webdriver.Chrome(options=options)
    # driver = webdriver.Edge(options=options)
    driver.set_window_size(1120, 1000)

    url = 'https://www.glassdoor.com/Job/jobs.htm?sc.keyword=' + keyword 
    # + '"&locT=C&locId=1147401&jobType=all&fromAge=-1&minSalary=0&includeNoSalaryJobs=true&radius=100&cityId=-1&minRating=0.0&industryId=-1&sgocId=-1&seniorityType=all&companyId=-1&employerSizes=0&applicationType=0&remoteWorkType=0'
    driver.get(url)
    
    time.sleep(2)
    
    try:
        WebDriverWait(driver, 10).until(EC.frame_to_be_available_and_switch_to_it((By.XPATH,"//iframe[@title='Widget containing a Cloudflare security challenge']")))
        WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, "//label[@class='ctp-checkbox-label']"))).click()
    except (NoSuchElementException, TimeoutException):
        pass
        
    jobs = []
    job_buttons = []
    startingIndex = 0
    
    # driver.minimize_window()
    driver.switch_to.default_content()
    # time.sleep(2)
    
    # driver.refresh()
    
    print(location)
    if location:
        try:
            # Wait for the location input field to be available
            location_input = WebDriverWait(driver, 60).until(
                # EC.presence_of_element_located((By.CSS_SELECTOR, "#searchBar-location"))
                EC.presence_of_element_located((By.XPATH, "//input[@id='searchBar-location']"))
            )
            location_input.clear()  # Clearing the field before entering new text
            location_input.send_keys(location)
            location_input.send_keys(Keys.ENTER)  # Pressing the Enter key
            time.sleep(2)
            # driver.refresh()
            # time.sleep(2)
        except TimeoutException:
            print("Location input field not found within the specified time")
            pass
        
    driver.switch_to.default_content()
    try:
        WebDriverWait(driver, 60).until(EC.visibility_of_element_located((By.CSS_SELECTOR, 'h1.SearchResultsHeader_jobCount__12dWB')))
    except (NoSuchElementException, TimeoutException):
        pass
    
    num_jobs_found = int(re.sub(r'[^\d.]', '', driver.execute_script("return document.querySelector('h1.SearchResultsHeader_jobCount__12dWB').innerText;")))
    print(num_jobs_found,'total jobs found')
    
    if num_jobs_found > 900:
        print('WARNING: More than 900 jobs found. At most 900 are retrievable at once.')
    
    # if not num_jobs or num_jobs < 0:
    #     print('num_jobs not specified. Getting all jobs...')
    #     # num_jobs = num_jobs_found + 100 # upper bounding just in case
    #     num_jobs = min(900, num_jobs_found) # 6000
    # else:
    #     print('Getting the first',str(num_jobs),'jobs...')

    # pbar = tqdm(total=num_jobs)
    # while len(jobs) < num_jobs:
    pbar = tqdm()
    while True:
        
        # Take the current snapshot of job buttons
        current_count = len(job_buttons)
        
        # Wait until new job list items are loaded or until a timeout
        try:
            WebDriverWait(driver, 40, 1).until(
                lambda d: d.execute_script("return document.querySelectorAll('li.JobsList_jobListItem__JBBUV').length;") > current_count
            )
        except TimeoutException:
            print("No more jobs were loaded. Ending scraping with {} jobs.".format(len(jobs)))
            break

        # Retrieve the updated list of job buttons
        job_buttons = driver.execute_script("return document.querySelectorAll('li.JobsList_jobListItem__JBBUV');")

        # Check if new jobs have been loaded
        if len(job_buttons) > current_count:
            if verbose:
                print(f"{len(job_buttons) - current_count} new jobs found, processing...")
        else:
            print("No more jobs were loaded. Ending scraping with {} jobs.".format(len(jobs)))
            break
        
        
        ##################

        for job_button in job_buttons[startingIndex:]:
            # if len(jobs) >= num_jobs:
            #     break

            driver.execute_script("arguments[0].click();", job_button)
            # time.sleep(0.4)

            try:
                company_name = job_button.find_element(By.CSS_SELECTOR, 'div.EmployerProfile_employerInfo__GaPbq').text
            except NoSuchElementException:
                company_name = ''
                continue
            try:
                rating = job_button.find_element(By.CSS_SELECTOR, 'span.EmployerProfile_employerRating__3ADTJ').text
                if rating != '':
                    company_name = company_name[:-len(rating)]
                    rating = float(re.sub(r'[^\d.]', '', rating))
                else:
                    rating = -1
            except NoSuchElementException:
                rating = -1
            try:
                salary_estimate = job_button.find_element(By.CSS_SELECTOR, 'div.JobCard_salaryEstimate___m9kY').text
            except NoSuchElementException:
                salary_estimate = -1
                
            try:
                # WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.CSS_SELECTOR, 'div.JobDetails_location__MbnUM')))
                # WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.CSS_SELECTOR, 'div.JobDetails_jobTitle__Rw_gn')))
                WebDriverWait(driver, 60).until(EC.visibility_of_element_located((By.CSS_SELECTOR, 'div.JobDetails_jobDescription__6VeBn')))
                location = driver.execute_script("return document.querySelector('div.JobDetails_location__MbnUM').innerText;")
                job_title = driver.execute_script("return document.querySelector('div.JobDetails_jobTitle__Rw_gn').innerText;")
                job_description = driver.execute_script("return document.querySelector('div.JobDetails_jobDescription__6VeBn').innerText;")

            except Exception as e:
                print(f"Failed to collect data for a job due to: {e}")
                continue
            

            company_data = driver.execute_script(company_script)
            # print(company_data)

            if verbose:
                print("Job Title: {}".format(job_title))
                # print("Salary Estimate: {}".format(salary_estimate))
                # print("Job Description: {}".format(job_description[:500]))
                # print("Rating: {}".format(rating))
                print("Company Name: {}".format(company_name))
                # print("Location: {}".format(location))
                
            items = {
                "Job Title": job_title,
                "Company Name": company_name,
                "Location": location,
                "Salary Estimate": salary_estimate,
                "Rating": rating,
                "Job Description": job_description,
            }
            items.update(company_data)
            items.update({"Easy Apply" : driver.execute_script(easy_apply_script)})
            jobs.append(items)
            pbar.update(1)
                
        ##################

        # Attempt to click the 'load more' button to load additional jobs
        # load_more_button_xpath = "//button[.//span[contains(text(),'Show more jobs')]]"
        # try:
        #     WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, load_more_button_xpath)))
        #     driver.execute_script("arguments[0].click();", driver.find_element(By.XPATH, load_more_button_xpath))
        # except (NoSuchElementException, TimeoutException):
        #     print("No 'Show more jobs' button found. Ending scraping with {} jobs.".format(len(jobs)))
        #     break
        
        load_more_button_xpath = "#left-column > div.JobsList_wrapper__wgimi > div > button"
        try:
            WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.CSS_SELECTOR, load_more_button_xpath)))
            driver.execute_script("arguments[0].click();", driver.find_element(By.CSS_SELECTOR, load_more_button_xpath))
        except (NoSuchElementException, TimeoutException):
            print("No 'Show more jobs' button found. Ending scraping with {} jobs.".format(len(jobs)))
            break
        
        # Update startingIndex to the length of the current job_buttons for the next iteration
        startingIndex = len(job_buttons)-1
    
    pbar.close()
    
    # print('ending len(job_buttons):',len(job_buttons))
    # print(len(jobs), num_jobs, num_jobs_found)
    # print(len(jobs), num_jobs_found)
    
    
    return pd.DataFrame(jobs)  #This line converts the dictionary object into a pandas DataFrame.

In [33]:
# #This line will open a new chrome window and start the scraping.
# keyword = "data analyst"
# # df = get_jobs(keyword, 60, False)
# df = get_jobs(keyword, 'Illinois, US', None, False)
# pd.DataFrame(df).to_csv(os.path.join('scraper results test',keyword+' '+str(len(df))+'.csv'), index=False)
# # df

### Get results df from all states

In [34]:
def get_jobs_states(keyword):
    path = os.path.join('scraper results', f'{keyword} state results')
    if not os.path.isdir(path):
        os.mkdir(path)
    for state in us_states:
        df = get_jobs(keyword, f'{state}, US', None, False)
        pd.DataFrame(df).to_csv(os.path.join(path, f'{keyword} in {state} {str(len(df))}.csv'), index=False)
        # df

In [35]:
# get_jobs_states("data scientist")

In [36]:
# get_jobs_states("data analyst")

In [37]:
# get_jobs_states("business analyst")

In [38]:
# get_jobs_states("data engineer")

loop through the list of all states with <= 900 postings, saving a dataframe for each state

for states with more > 900 postings, create a separate loop for each, where you loop through all of its cities (glassdoor doesnt do counties)


finally, join all dataframes together, remove duplicates then save


In [39]:
def get_job_count(keyword, location=None, num_jobs=None, verbose=False):
    
    '''Gathers jobs as a dataframe, scraped from Glassdoor'''
    #Initializing the webdriver
    options = Options()
    options.page_load_strategy = 'normal'
    # options.add_experimental_option("detach", True)

    driver = webdriver.Chrome(options=options)
    driver.set_window_size(1120, 1000)

    url = 'https://www.glassdoor.com/Job/jobs.htm?sc.keyword=' + keyword 
    driver.get(url)
    
    # time.sleep(2)
    
    try:
        WebDriverWait(driver, 10).until(EC.frame_to_be_available_and_switch_to_it((By.XPATH,"//iframe[@title='Widget containing a Cloudflare security challenge']")))
        WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, "//label[@class='ctp-checkbox-label']"))).click()
    except (NoSuchElementException, TimeoutException):
        pass
    
    driver.minimize_window()
    driver.switch_to.default_content()
    # time.sleep(2)
    
    if location:
        try:
            # Wait for the location input field to be available
            location_input = WebDriverWait(driver, 60).until(
                # EC.presence_of_element_located((By.CSS_SELECTOR, "#searchBar-location"))
                EC.presence_of_element_located((By.XPATH, "//input[@id='searchBar-location']"))
            )
            location_input.clear()  # Clearing the field before entering new text
            location_input.send_keys(location)
            location_input.send_keys(Keys.ENTER)  # Pressing the Enter key
            time.sleep(2)
            
        except TimeoutException:
            print("Location input field not found within the specified time")
            pass
    driver.switch_to.default_content()
    try:
        WebDriverWait(driver, 60).until(EC.visibility_of_element_located((By.CSS_SELECTOR, 'h1.SearchResultsHeader_jobCount__12dWB')))
    except (NoSuchElementException, TimeoutException):
        pass
    
    num_jobs_found = int(re.sub(r'[^\d.]', '', driver.execute_script("return document.querySelector('h1.SearchResultsHeader_jobCount__12dWB').innerText;")))
    return num_jobs_found

In [40]:
def save_dict_as_df(d, keyword):
    df = pd.DataFrame().from_dict(d.items())
    df.columns=['State','Postings']
    df.to_csv( os.path.join('state counts', f'{keyword} state counts.csv'),index=False)

In [41]:
# keyword = "data scientist"
# counts_per_state = dict()

# for state in us_states:
#     c = get_job_count(keyword, f'{state}, US', None, False)
#     counts_per_state[state] = c
#     print(state, c)
# save_dict_as_df(counts_per_state, keyword)
# counts_per_state

In [42]:
# keyword = "data analyst"
# counts_per_state = dict()

# for state in us_states:
#     c = get_job_count(keyword, f'{state}, US', None, False)
#     counts_per_state[state] = c
#     print(state, c)
# save_dict_as_df(counts_per_state, keyword)
# counts_per_state

In [43]:
# keyword = "business analyst"
# counts_per_state = dict()

# for state in us_states:
#     c = get_job_count(keyword, f'{state}, US', None, False)
#     counts_per_state[state] = c
#     print(state, c)
# save_dict_as_df(counts_per_state, keyword)
# counts_per_state

In [44]:
# keyword = "data engineer"
# counts_per_state = dict()

# for state in us_states:
#     c = get_job_count(keyword, f'{state}, US', None, False)
#     counts_per_state[state] = c
#     print(state, c)
# save_dict_as_df(counts_per_state, keyword)
# counts_per_state

In [45]:
for keyword in ['data scientist', 'data analyst', 'business analyst', 'data engineer']:
    fnames = set(os.listdir(os.path.join('scraper results', f'{keyword} state results')))
    for state in us_states:
        fname = [a for a in fnames if state in a][0]
        counts = pd.read_csv(os.path.join('scraper results', f'{keyword} state results',fname))
        if len(counts) > 800:
            print(fname)
        # df

data scientist in California 900.csv


data analyst in California 900.csv
data analyst in Texas 900.csv
data analyst in Washington, DC 900.csv
business analyst in California 900.csv
business analyst in Texas 898.csv


### Extract postigs from city level for the states that have more than the 900 postings limit

In [46]:
# 'data scientist'
# ["CA", "WA"]
# 'data analyst'
# ["CA", "TX", "WA"]
# 'business analyst'
# ["CA", "TX", "WA"]


In [47]:
def get_jobs_state_cities(keyword, state, cities):
    if not os.path.isdir(os.path.join(f'{keyword} state results', f'{keyword} in {state} results')):
        os.mkdir(os.path.join(f'{keyword} state results', f'{keyword} in {state} results'))
    for city in cities:
        if len([a for a in os.listdir(os.path.join(f'{keyword} state results', f'{keyword} in {state} results')) if city in a]) == 1:
            continue
        df = get_jobs(keyword, f'{city}, {state}', None, False)
        pd.DataFrame(df).to_csv(os.path.join(f'{keyword} state results', f'{keyword} in {state} results',f'{keyword} in {city}, {state} {str(len(df))}.csv'), index=False)

data scientist

In [48]:
# keyword, state = 'data scientist', "CA"
# cities = ['Cupertino','Los Angeles','Menlo Park','Mountain View','Palo Alto','San Diego','San Francisco','San Jose','Santa Clara','Sunnyvale']
# get_jobs_state_cities(keyword, state, cities)

data analyst

In [49]:
# keyword, state = 'data analyst', "CA"
# cities = ['El Segundo','Irvine','Los Angeles','Mountain View','Oakland','Sacramento','San Diego','San Francisco','San Jose','Santa Clara']

# get_jobs_state_cities(keyword, state, cities)

In [50]:
# keyword, state = 'data analyst', "TX"
# cities = ['Austin','Dallas','El Paso','Fort Worth','Frisco','Houston','Irving','Plano','Richardson','San Antonio']
# get_jobs_state_cities(keyword, state, cities)

business analyst

In [51]:
# keyword, state = 'business analyst', "CA"
# cities = ['Burbank','Folsom','Irvine','Los Angeles','Oakland','Sacramento','San Diego','San Francisco','San Jose','Santa Clara']
# get_jobs_state_cities(keyword, state, cities)

In [52]:
# keyword, state = 'business analyst', "TX"
# cities = ['Austin','Dallas','El Paso','Fort Worth','Frisco','Houston','Irving','Plano','Richardson','San Antonio']
# get_jobs_state_cities(keyword, state, cities)

In [53]:
def remove_duplicates(df:pd.DataFrame, exclude=None):
    
    subset_columns = ['Job Title', 'Company Name', 'Location', 'Salary Estimate', 'Rating', 'Job Description']
    
    if exclude:
        subset_columns.remove(exclude)
    
    df.sort_values(subset_columns, inplace=True)

    df['Duplicate Count'] = df.groupby(subset_columns)['Job Title'].transform('count')
    df.drop_duplicates(subset=subset_columns, inplace=True, keep='last')  
    # combined_data.drop_duplicates(inplace=True, keep='last')  
    

In [54]:
def combine_results(dirname, exclude=None):
    dataframes = []
    for root, _, files in os.walk(dirname):
        for file in files:
            if file.endswith('.csv'):
                file_path = os.path.join(root, file)
                data = pd.read_csv(file_path)
                dataframes.append(data)
    combined_data = pd.concat(dataframes, ignore_index=True)
    
    remove_duplicates(combined_data, exclude)

    return combined_data


more rows remaining = less duplicates

In [55]:
# test = [None, 'Job Title', 'Company Name', 'Location', 'Salary Estimate', 'Rating','Job Description']
# total = 0

# for i in ['business analyst', 'data analyst', 'data scientist', 'data engineer']:
#     print('\n',i)
#     for exclude_col in test:
#         d = combine_results(os.path.join('scraper results', f'{i} state results'), exclude_col)
#         # print(f'{exclude_col}\t       \t{d.shape[0]}')
#         if exclude_col is None:
#             total = d.shape[0]
#             print(f'{exclude_col}\t       \t{total}')
#         else:
#             print(f'{exclude_col}\t       \t{d.shape[0]}\t-{total-d.shape[0]}\t-{100*(total-d.shape[0])/total:.2f}%')

- Location differences cause the most repeat postings (this is expected)
- Different job descriptions causing the next most repeat postings.
  - Question - should I remove different job descriptions for the same job when doing skill analysis?
    - effective is small, <= 1% --> no, keep them
  - Question - should I remove different locations for the same job when doing skill analysis?
    - effective is ~ 4-9% --> yes, remove them
    - combine_results(exclude='Location')


In [56]:
for i in ['business analyst', 'data analyst', 'data scientist', 'data engineer']:
    print(i)
    d = combine_results(os.path.join('scraper results', f'{i} state results'))
    print(d.shape)
    d.to_csv(os.path.join('combined scraper results', f'{i.replace("state ","")}.csv') ,index=False)

business analyst
(10830, 14)
(10411, 14)
data analyst
(9923, 14)
(9560, 14)
data scientist
(5539, 14)
(5023, 14)
data engineer
(2465, 14)
(2304, 14)
